# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 50 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.00006


extracting tarball to tmp_2209.00006...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.00009


extracting tarball to tmp_2209.00009...

...

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2209.00620/submit.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure identification_plots/NIRSpec_G140H-F100LP.jpg
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/tmp/ipykernel_2241/4030337529.py:34: LatexWarning: 2209.00620 did not run properly
Could not find figure identification_plots/NIRSpec_G140H-F100LP.jpg
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2209.00635


extracting tarball to tmp_2209.00635...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00363-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00363) | **WISDOM Project -- XIII. Feeding molecular gas to the supermassive black  hole in the starburst AGN-host galaxy Fairall 49**  |
|| Federico Lelli, et al. -- incl., <mark>Mark D. Smith</mark>, <mark>Thomas G. Williams</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *19 pages, 13 figures, accepted for publication in MNRAS*|
|**Abstract**| The mm-Wave Interferometric Survey of Dark Object Masses (WISDOM) is probing supermassive black holes (SMBHs) in galaxies across the Hubble sequence via molecular gas dynamics. We present the first WISDOM study of a luminous infrared galaxy with an active galactic nuclei (AGN): Fairall 49. We use new ALMA observations of the CO(2-1) line with a spatial resolution of about 80 pc together with ancillary HST imaging. We reach the following results: (1) The CO kinematics are well described by a regularly rotating gas disk with a radial inflow motion, suggesting weak feedback on the cold gas from both AGN and starburst activity; (2) The dynamically inferred SMBH mass is 1.6 +/- 0.4 (rnd) +/- 0.8 (sys) x 10^8 Msun, assuming that we have accurately subtracted the AGN and starburst light contributions, which have a luminosity of about 10^9 Lsun; (3) The SMBH mass agrees with the SMBH-stellar mass relation but is about 50 times higher than previous estimates from X-ray variability; (4) The dynamically inferred molecular gas mass is 30 times smaller than that inferred from adopting the Galactic CO-to-H_2 conversion factor (X_CO) for thermalised gas, suggesting low values of X_CO; (5) the molecular gas inflow rate increases steadily with radius and may be as high as 5 Msun/yr. This work highlights the potential of using high-resolution CO data to estimate, in addition to SMBH masses, the X_CO factor and gas inflow rates in nearby galaxies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00018-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00018) | **A fast rising tidal disruption event from a candidate intermediate mass  black hole**  |
|| C. R. Angus, et al. -- incl., <mark>V. A. Villar</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *Accepted for publication in Nature Astronomy*|
|**Abstract**| Massive black holes (BHs) at the centres of massive galaxies are ubiquitous. The population of BHs within dwarf galaxies, on the other hand, is evasive. Dwarf galaxies are thought to harbour BHs with proportionally small masses, including intermediate mass BHs, with masses $10^{2} < M_{BH} < 10^{6} M_{\odot}$. Identification of these systems has historically relied upon the detection of light emitted from accreting gaseous discs close to the BHs. Without this light, they are difficult to detect. Tidal disruption events (TDEs), the luminous flares produced when a star strays close to a BH and is shredded, are a direct way to probe massive BHs. The rise times of these flares theoretically correlate with the BH mass. Here we present AT2020neh, a fast rising TDE candidate, hosted by a dwarf galaxy. AT2020neh can be described by the tidal disruption of a main sequence star by a 10$^{4.7} - 10^{5.9} M_{\odot}$ BH. We find the observable rate of fast rising nuclear transients like AT2020neh to be rare, at $\lesssim 2 \times 10^{-8}$ events Mpc$^{-3}$ yr$^{-1}$. Finding non-accreting BHs in dwarf galaxies is important to determine how prevalent BHs are within these galaxies, and constrain models of BH formation. AT2020neh-like events may provide a galaxy-independent method of measuring IMBH masses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00006-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00006) | **A Phantom Menace: On the Morphology of the Galactic Center Excess**  |
|| Samuel D. McDermott, <mark>Yi-Ming Zhong</mark>, Ilias Cholis |
|*Appeared on*| *2022-09-02*|
|*Comments*| *5 pages, code available at this https URL*|
|**Abstract**| The characteristics of the Galactic Center Excess (GCE) emission observed in gamma-ray energies -- especially the morphology of the GCE -- remain a hotly debated subject. The manner in which the dominant diffuse gamma-ray background is modeled has been claimed to have a determining effect on the preferred morphology. In this work, we compare two distinct approaches to the galactic diffuse gamma-ray emission background: the first approach models this emission through templates calculated from a sequence of well-defined astrophysical assumptions, while the second approach divides surrogates for the background gamma-ray emission into cylindrical galactocentric rings with free independent normalizations. At the latitudes that we focus on, we find that the former approach works better, and that the overall best fit is obtained for an astrophysically motivated fit when the GCE follows the morphology expected of dark matter annihilation. Quantitatively, the improvement compared to the best ring-based fits is roughly 6500 in the chi^2 and roughly 4000 in the log of the Bayesian evidence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00432-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00432) | **Snapshot of a magnetohydrodynamic disk wind traced by water maser  observations**  |
|| Luca Moscadelli, et al. -- incl., <mark>Henrik Beuther</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *Main paper: 15 pages, 6 figures, 2 tables; Appendix: 21 pages, 7 figures, 1 table*|
|**Abstract**| The formation of astrophysical objects of different nature and size, from black holes to gaseous giant planets, involves a disk-jet system, where the disk drives the mass accretion onto a central compact object and the jet is a fast collimated ejection along the disk rotation axis. Magnetohydrodynamic disk winds can provide the link between mass accretion and ejection, which is essential to ensure that the excess angular momentum is removed from the system and accretion onto the central object can proceed. However, up to now, we have been lacking direct observational proof of disk winds. This work presents a direct view of the velocity field of a disk wind around a forming massive star. Achieving a very high spatial resolution of ~0.05 au, our water maser observations trace the velocities of individual streamlines emerging from the disk orbiting the forming star. We find that, at low elevation above the disk midplane, the flow co-rotates with its launch point in the disk, in agreement with magneto-centrifugal acceleration where the gas is flung away along the magnetic field line anchored to the disk. Beyond the co-rotation point, the flow rises spiraling around the disk rotation axis along a helical magnetic field. We have performed (resistive-radiative-gravito-) magnetohydrodynamic simulations of the formation of a massive star and record the development of a magneto-centrifugally launched jet presenting many properties in agreement with our observations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure combomap.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00620-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00620) | **The JWST Early Release Science Program for Direct Observations of  Exoplanetary Systems II: A 1 to 20 Micron Spectrum of the Planetary-Mass  Companion VHS 1256-1257 b**  |
|| Brittany E. Miles, et al. -- incl., <mark>Alexandra Z. Greenbaum</mark>, <mark>Thomas Henning</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *Submitted to AAS Journals*|
|**Abstract**| We present the highest fidelity spectrum to date of a planetary-mass object. VHS 1256 b is a $<$20 M$_\mathrm{Jup}$ widely separated ($\sim$8", a = 150 au), young, brown dwarf companion that shares photometric colors and spectroscopic features with the directly imaged exoplanets HR 8799 c, d, and e. As an L-to-T transition object, VHS 1256 b exists along the region of the color-magnitude diagram where substellar atmospheres transition from cloudy to clear. We observed VHS 1256 b with JWST's NIRSpec IFU and MIRI MRS modes for coverage from 1 $\mu$m to 20 $\mu$m at resolutions of $\sim$1,000 - 3,700. Water, methane, carbon monoxide, carbon dioxide, sodium, and potassium are observed in several portions of the JWST spectrum based on comparisons from template brown dwarf spectra, molecular opacities, and atmospheric models. The spectral shape of VHS 1256 b is influenced by disequilibrium chemistry and clouds. We directly detect silicate clouds, the first such detection reported for a planetary-mass companion. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure identification_plots/NIRSpec_G140H-F100LP.jpg</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00112-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00112) | **KMT-2017-BLG-0673Lb and KMT-2019-BLG-0414Lb: Two microlensing planets  detected in peripheral fields of KMTNet survey**  |
|| Cheongho Han, et al. -- incl., <mark>Chung-Uk Lee</mark>, <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *9 pages, 7 figures*|
|**Abstract**| We investigate the microlensing data collected during the 2017--2019 seasons in the peripheral Galactic bulge fields with the aim of finding planetary signals in microlensing light curves observed with relatively sparse coverage. We first sort out lensing events with weak short-term anomalies in the lensing light curves from the visual inspection of all non-prime-field events, and then test various interpretations of the anomalies. From this procedure, we find two previously unidentified candidate planetary lensing events KMT-2017-BLG-0673 and KMT-2019-BLG-0414. It is found that the planetary signal of KMT-2017-BLG-0673 was produced by the source crossing over a planet-induced caustic, but it was previously missed because of the sparse coverage of the signal. On the other hand, the possibly planetary signal of KMT-2019-BLG-0414 was generated without caustic crossing, and it was previously missed due to the weakness of the signal. We identify a unique planetary solution for KMT-2017-BLG-0673. However, for KMT-2019-BLG-0414, we identify two pairs of planetary solutions, for each of which there are two solutions caused by the close-wide degeneracy, and a slightly less favored binary-source solution, in which a single lens mass gravitationally magnified a rapidly orbiting binary source with a faint companion (xallarap). From Bayesian analyses, it is estimated that the planet KMT-2017-BLG-0673Lb has a mass of $3.7^{+2.2}_{-2.1}~M_{\rm J}$, and it is orbiting a late K-type host star with a mass of $0.63^{+0.37}_{-0.35}~M_\odot$. Under the planetary interpretation of KMT-2010-BLG-0414L, a star with a mass of $0.74^{+0.43}_{-0.38}~M_\odot$ hosts a planet with a mass of $\sim 3.2$--3.6~$M_{\rm J}$ depending on the solution. We discuss the possible resolution of the planet-xallarap degeneracy of KMT-2019-BLG-0414 by future adaptive-optics observations on 30~m class telescopes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00549-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00549) | **The evolving absolute magnitude of type 1a supernovae and its critical  impact on the cosmological parameters**  |
|| <mark>A.P. Mahtessian</mark>, G.S. Karapetian, M. A. Hovhannisyan, <mark>L.A. Mahtessian</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *17 pages, 8 figures, 12 tables*|
|**Abstract**| In this work, a computer optimization model has been developed that allows one to load the initial observation data of supernovae 1a into a table and, simply, by searching for the best fit between observations and theory, obtain the values of the parameters of cosmological models. Naturally, the initial data are redshifts z and apparent magnitudes m at the maximum brightness of supernovae. For better fit between theory and observation, Pearson's Chi2 (Chi-squared) goodness-of-fit test was used. The results are obtained for the LCDM model and, for comparison, the model with a zero cosmological constant. In order to improve the fit between observed data and theory, the optimization is carried out assuming that the absolute magnitude of supernovae is not constant, but evolves with time. It is assumed that the dependence of the absolute magnitude on the redshift is linear: M=M(z=0)+ez, where e is the evolution coefficient of the absolute magnitude of type 1a supernovae. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.00028-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.00028) | **The iLocater cryostat and thermal control system: enabling extremely  precise radial velocity measurements for diffraction-limited spectrographs**  |
|| Jonathan Crass, et al. -- incl., <mark>Frederick R. Hearty</mark> |
|*Appeared on*| *2022-09-02*|
|*Comments*| *10 pages, In proceedings of SPIE 12184*|
|**Abstract**| Extremely precise radial velocity (EPRV) measurements are critical for characterizing nearby terrestrial worlds. EPRV instrument precisions of $\sigma_{\mathrm{RV}} = 1-10\,\mathrm{cm/s}$ are required to study Earth-analog systems, imposing stringent, sub-mK, thermo-mechanical stability requirements on Doppler spectrograph designs. iLocater is a new, high-resolution ($R=190,500$ median) near infrared (NIR) EPRV spectrograph under construction for the dual 8.4 m diameter Large Binocular Telescope (LBT). The instrument is one of the first to operate in the diffraction-limited regime enabled by the use of adaptive optics and single-mode fibers. This facilitates affordable optomechanical fabrication of the spectrograph using intrinsically stable materials. We present the final design and performance of the iLocater cryostat and thermal control system which houses the instrument spectrograph. The spectrograph is situated inside an actively temperature-controlled radiation shield mounted inside a multi-layer-insulation (MLI) lined vacuum chamber. The radiation shield provides sub-mK thermal stability, building on the existing heritage of the Habitable-zone Planet Finder (HPF) and NEID instruments. The instrument operating temperature ($T=80-100\,\mathrm{K}$) is driven by the requirement to minimize detector background and instantaneous coefficient of thermal expansion (CTE) of the materials used for spectrograph fabrication. This combination allows for a reduced thermomechanical impact on measurement precision, improving the scientific capabilities of the instrument. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error unexpected end of data</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.00363.md
    + _build/html/tmp_2209.00363/./FRL49_HST_Inn.png
    + _build/html/tmp_2209.00363/./FRL49_HST_Out.png
    + _build/html/tmp_2209.00363/./FRL49_MCMC_sub2.jpg
    + _build/html/tmp_2209.00363/./FRL49_HSTprof.jpg


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{\ensuremath{\mathrm{km} \mathrm{s}^{-1}}}$
$\newcommand{\MLsun}{\ensuremath{\mathrm{M}_{\odot}/\mathrm{L}_{\odot}}}$
$\newcommand{\Lsun}{\ensuremath{\mathrm{L}_{\odot}}}$
$\newcommand{\Msun}{\ensuremath{\mathrm{M}_{\odot}}}$
$\newcommand{\lumdens}{\ensuremath{\mathrm{L}_{\odot} \mathrm{pc}^{-2}}}$
$\newcommand{\surfdens}{\ensuremath{\mathrm{M}_{\odot} \mathrm{pc}^{-2}}}$
$\newcommand{\XCO}{\ensuremath{X_\mathrm{CO}}}$
$\newcommand{\hi }{{\rm H} {\small\rm I}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{$\ensuremath${\mathrm{km} \mathrm{s}^{-1}}}$
$\newcommand{$\MLsun$}{$\ensuremath${\mathrm{M}_{\odot}/\mathrm{L}_{\odot}}}$
$\newcommand{$\Lsun$}{$\ensuremath${\mathrm{L}_{\odot}}}$
$\newcommand{$\Msun$}{$\ensuremath${\mathrm{M}_{\odot}}}$
$\newcommand{$\lumdens$}{$\ensuremath${\mathrm{L}_{\odot} \mathrm{pc}^{-2}}}$
$\newcommand{$\surfdens$}{$\ensuremath${\mathrm{M}_{\odot} \mathrm{pc}^{-2}}}$
$\newcommand{$\XCO$}{$\ensuremath${X_\mathrm{CO}}}$
$\newcommand{$\hi$ }{{\rm H} {\small\rm I}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# WISDOM Project - XIII. Feeding molecular gas to the supermassive black hole in the starburst AGN-host galaxy Fairall 49

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.00363-b31b1b.svg)](https://arxiv.org/abs/2209.00363)<mark>Appeared on: 2022-09-02</mark> - _19 pages, 13 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

Federico Lelli, et al. -- incl., <mark><mark>Mark D. Smith</mark></mark>, <mark><mark>Thomas G. Williams</mark></mark>

</div>
<div id="abstract">

**Abstract:** The mm-Wave Interferometric Survey of Dark Object Masses (WISDOM) is probing supermassive black holes (SMBHs) in galaxies across the Hubble sequence via molecular gas dynamics. We present the first WISDOM study of a luminous infrared galaxy with an active galactic nuclei (AGN): Fairall 49. We use new ALMA observations of the CO($2-1$) line with a spatial resolution of$\sim$80 pc together with ancillary HST imaging. We reach the following results: (1) The CO kinematics are well described by a regularly rotating gas disk with a radial inflow motion, suggesting weak feedback on the cold gas from both AGN and starburst activity; (2) The dynamically inferred SMBH mass is$1.6\pm0.4\mathrm{(rnd)}\pm0.8 \mathrm{(sys)}\times 10^{8}$\Msunassuming that we have accurately subtracted the AGN and starburst light contributions, which have a luminosity of$\sim$10$^9$L$_\odot$; (3) The SMBH mass agrees with the SMBH$-$stellar mass relation but is$\sim$50 times higher than previous estimates from X-ray variability; (4) The dynamically inferred molecular gas mass is 30 times smaller than that inferred from adopting the Galactic CO-to-H$_2$conversion factor ($\XCO$) for thermalised gas, suggesting low values of$\XCO$; (5) the molecular gas inflow rate increases steadily with radius and may be as high as$\sim$5$M_\odot$yr$^{-1}$. This work highlights the potential of using high-resolution CO data to estimate, in addition to SMBH masses, the X$_{\rm CO}$factor and gas inflow rates in nearby galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2209.00363/./FRL49_HST_Inn.png" alt="Fig6.1" width="50%"/><img src="tmp_2209.00363/./FRL49_HST_Out.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** HST/WFPC2 image of Fairall 49 in the F606W filter. The two panels show the same HST image but use a different color stretching to highlight different features in the galaxy. The left panel shows the inner spiral disk, while the right panel highlights the outer asymmetric stellar distribution superimposed with the CO(2-1) emission (cyan contours). Contours increase from 0.14 ($\sim$3$\sigma$) to 8 mJy beam$^{-1}$ km s$^{-1}$. The bar to the bottom-right corner equals $\sim$2.5$"$, corresponding to $\sim$1 kpc for the assumed distance of 86.7 Mpc. North is up; East is left. (*fig:HST*)

</div>
<div id="div_fig2">

<img src="tmp_2209.00363/./FRL49_MCMC_sub2.jpg" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{Top panel:} The observed rotation curve (black dots with errorbars) is fitted with a mass model (blue solid line) that considers the gravitational contributions of stars (red dashed line), gas (green dotted line), black hole (black dash-dotted line) and DM halo (purple dash-dotted line). \textit{Bottom panel:} gas inflow rate considering the molecular gas mass from the mass model. The error bars comprise the uncertainties on $V_{\rm rad}$ only; the uncertainties on $M_{\rm gas}$ have a systematic effect, shifting the inner peak up and down (see Eq. \ref{eq:inflow}). (*fig:massmodel*)

</div>
<div id="div_fig3">

<img src="tmp_2209.00363/./FRL49_HSTprof.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** Surface brightness profile of Fairall 49 from the HST/WFPC2 image in the F606W filter. The dashed line shows an exponential fit to the outer parts. The inset zooms at small radii ($R<0.7$ kpc): the blue circles corresponds to the inner HSB component that is likely dominated by the AGN and/or starburst emission, the red circles shows the inward exponential extrapolation of the stellar component. See text for details. (*fig:sbp*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

22  publications in the last 7 days.
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers